# 끄적거리거나 테스트 하는 코드

```
efficientnet_b4 : 19M
coatnet_0_rw_224 : 27M  
swin_s3_tiny_224 : 28M
- convnext  
- convmixer  
```

In [4]:
import torch
import torch.nn as nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import torch.nn.functional as F

import random

import numpy as np

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score

import multiprocessing
import timm

from PIL import Image

import os
import json

import pandas as pd
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

from sklearn.model_selection import train_test_split

In [5]:
print(pd.Series([1,0,2]).to_numpy(np.float32), pd.Series([1,0,2]).to_numpy(np.float32).dtype)

[1. 0. 2.] float32


In [6]:
class CustomTrainDataset(Dataset):
    ## input pd.Series
    ## output np.ndarray
    ## change dummy, if label_col is 'gender' or 'mask_state'
    def __init__(self, 
                 img_paths : pd.Series, 
                 mask_labels : pd.Series, 
                 gender_labels:pd.Series, 
                 age_labels:pd.Series, 
                 transforms=None):
        self.img_paths = img_paths.to_numpy()
        self.transforms = transforms
        self.mask_labels = mask_labels.to_numpy(np.float32)
        self.gender_labels = gender_labels.to_numpy(np.float32)
        self.age_labels = age_labels.to_numpy(np.float32)
        assert self.transforms != None, 'you must use transforms in Trainset'
    
    ## return numpy img, label
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = np.array(Image.open(img_path))

        img = self.transforms(image=img)["image"]
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]

        return img, mask_label, gender_label, age_label

    def __len__(self):
        return len(self.img_paths)

In [7]:
train_transforms = A.Compose([
        A.Resize(224, 224),
        A.CLAHE(),
        A.Normalize(mean=(0.560,0.524,0.501), std=(0.233,0.243,0.246)),
        ToTensorV2()
    ])

In [8]:
df = pd.read_csv(os.path.join(os.getcwd(), 'dataset', 'train', '3head_train.csv'))
dset = CustomTrainDataset(df['path'], df['mask_class'], df['gender_class'], df['age'], transforms = train_transforms)

In [18]:
df

,Unnamed: 0,path,mask_class,gender_class,age,origin_label
0,0,/opt/ml/repo/level1_imageclassification_cv-lev...,0,1,45,4
1,1,/opt/ml/repo/level1_imageclassification_cv-lev...,2,1,45,16
2,2,/opt/ml/repo/level1_imageclassification_cv-lev...,1,1,45,10
3,3,/opt/ml/repo/level1_imageclassification_cv-lev...,0,1,45,4
4,4,/opt/ml/repo/level1_imageclassification_cv-lev...,0,1,45,4
...,...,...,...,...,...,...
18895,18895,/opt/ml/repo/level1_imageclassification_cv-lev...,1,0,19,6
18896,18896,/opt/ml/repo/level1_imageclassification_cv-lev...,0,0,19,0
18897,18897,/opt/ml/repo/level1_imageclassification_cv-lev...,0,0,19,0
18898,18898,/opt/ml/repo/level1_imageclassification_cv-lev...,0,0,19,0


In [9]:
img, mask, gender, age = dset[1]

In [10]:
train_loader = DataLoader(dset,batch_size=8)

In [12]:
a,b,c,d = next(iter(train_loader))

In [17]:
print(a.dtype, b.dtype, c.dtype, d.dtype)

torch.float32 torch.float32 torch.float32 torch.float32


In [4]:
!nvidia-smi

Sat Oct 29 17:49:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/A   54C    P0    99W / 250W |  19467MiB / 32510MiB |     71%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------